In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from CustomTransform import CustomTransform
from VanillaNet import VanillaNet
from FeatureNet import FeatureNet

In [ ]:
batch_size = 32
loaders = 0
epochs = 64
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

custom_features = False;

In [ ]:
standard_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

custom_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     CustomTransform()])
net = None
super_tag = None

if custom_features:
    transform = custom_transform
    net = FeatureNet()
    super_tag = '_custom_'
else:
    # transform = standard_transform
    transform = custom_transform
    net = VanillaNet()
    super_tag = '_standard_'

In [ ]:
# Use this to download dataset
path_to_dataset = 'C:/Users/Aakash/Workspace/Dataset/CIFAR10'
trainset = torchvision.datasets.CIFAR10(root=path_to_dataset, train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root=path_to_dataset, train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=loaders)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=loaders)

In [ ]:
# Loss function

import torch.optim as optim
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is NOT available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    net = net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def train_network():
    global net
    total_loss = 0
    net.train()
    batches_so_far = 0
    correct = 0
    for x, y in trainloader:
        
        if train_on_gpu:
            x = x.cuda()
            y = y.cuda()

        optimizer.zero_grad()
        output = net(x)
        _, predicted = torch.max(output.data, 1)
        correct += (predicted == labels).sum().item()
        loss = criterion(output, y)
        
        loss.backward()
        optimizer.step()        
        total_loss += loss.item()
        batches_so_far += 1
    
    if batches_so_far > 1 :
        total_loss /= batches_so_far # loss per batch
        correct /= batches_so_far # correct guesses per batch
    
    return total_loss, correct
    

In [ ]:
def test_network():
    global net
    total_loss = 0
    correct = 0
    net.eval()
    batches_so_far = 0
    with torch.no_grad():
        for x, y in testloader:
            
            if train_on_gpu:
                x = x.cuda()
                y = y.cuda()
                
            output = net(x)
            loss = criterion(output, y)
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == labels).sum().item()
            total_loss += loss.item()
            batches_so_far+=1
    
    if batches_so_far > 1 :
        total_loss /= batches_so_far
        correct /= batches_so_far
    
    return total_loss, correct


In [ ]:
# Run specefic variables
import time
#from torch.utils.tensorboard import SummaryWriter
from tensorboardX import SummaryWriter

model_id = super_tag + str(int(time.time())) 

print('model id id ', model_id)
path_state_dict = "checkpoints/state_dict_"+model_id+".pth"
path_model = "checkpoints/model_"+model_id+".pth"

writer = SummaryWriter(comment=model_id)

tag_train_loss = "train_loss"
tag_test_loss = "test_loss"

tag_train_acc = "train_acc"
tag_test_acc = "test_acc"

In [ ]:
from IPython.display import clear_output
lowest_test_loss = np.inf

for e in range(epochs):
    train_loss, train_acc = train_network()
    writer.add_scalar(tag_train_loss, train_loss, e)
    writer.add_scalar(tag_train_acc, train_acc, e)
    test_loss, test_acc = test_network()
    writer.add_scalar(tag_test_loss, test_loss, e)
    writer.add_scalar(tag_test_acc, test_acc, e)
    
    clear_output()
    print("epoch", e, "\ttrain loss ", train_loss, "\ttest loss ", test_loss )
    
    if test_loss < lowest_test_loss:
        lowest_test_loss = test_loss
        torch.save(net.state_dict(), path_state_dict)
        torch.save(net, path_model)

In [ ]:
# Load the best model 
print("Loading best model so far")
net.load_state_dict(path_state_dict)